In [1]:
import numpy as np
import pandas as pd
from brian2 import *

INFO       Cache size for target "cython": 1957 MB.
You can call "clear_cache('cython')" to delete all files from the cache or manually delete files in the "C:\Users\user\.cython\brian_extensions" directory. [brian2]


In [8]:
#creating data
df = pd.DataFrame(np.random.randint(0,2,size=(50, 4)), columns=['N1','N2','N3','out'])

In [9]:
df


,N1,N2,N3,out
0,0,1,1,0
1,0,1,0,0
2,1,1,0,1
3,1,1,1,0
4,1,1,0,1
5,1,0,1,1
6,1,0,0,0
7,1,0,0,1
8,1,1,1,1
9,0,1,0,0


In [10]:
arr=df.to_numpy()

In [2]:
#enviroment
q_val_n1=np.zeros((1,3)) #agent 1 : neuron 1
q_val_n2 = np.zeros((1,3)) #agent 2 : neuron 2
q_val_n3=np.zeros((1,3)) #agent 3 : neuron 3
q_val_n4 = np.zeros((1,3)) #agent 4 : neuron 4
q_val_n5 = np.zeros((1,3)) #agent 5 : neuron 5

In [11]:
#definition of action
action = [0,1,2]
# 0 -> increase
# 1 -> fix
# 2 -> decrease
arr[1,3] =1

In [4]:
#definition of reward:
reward = -2
def get_reward(input_index,real_output):
    if arr[input_index,3] == real_output:
        reward  = 100
        return reward
    else:
        reward = -1
        return reward

In [5]:
#define an epsilon greedy algorithm that will choose which action to take next (i.e., where to move next)
def get_next_action(q_val, epsilon):
  #if a randomly chosen value between 0 and 1 is less than epsilon, 
  #then choose the most promising value from the Q-table for this state.
    if np.random.random() < epsilon:
        return np.argmax(q_val)
    else: #choose a random action
        return np.random.randint(3)

In [13]:
#define training parameters
epsilon = 0.9 #the percentage of time when we should take the best action (instead of a random action)
gamma = 0.9 #discount factor for future rewards
alpha = 0.9 #the rate at which the AI agent should learn

# 1 ->choose input pattern 
# 2 ->choose an action : 0 -> increase weight   1 -> fix   2 -> decrease
# 3 ->performe the choosen action , tansition ot the next state
# 4 ->Receive the reward for moving to the new state, and calculate the temporal difference
# 5 ->Update the Q-value for the previous state and action pair.
# 6 ->If the new (current) state is a terminal state, go to #1. Else, go to #2.
W = 0.1 # inc or dec
weight = 0.0 #initial weight
W_n1=[] #last weight for n1
W_n2=[] #last weight for n2
W_n3=[] #last weight for n3
W_n4=[] #last weight for n4
W_n5=[] #last weight for n5

for episode in range(1):
    #get the starting location for this episode
    
    #enviroment
    q_val_n1=np.zeros((1,3)) #agent 1 : neuron 1
    q_val_n2 = np.zeros((1,3)) #agent 2 : neuron 2
    q_val_n3=np.zeros((1,3)) #agent 3 : neuron 3
    q_val_n4 = np.zeros((1,3)) #agent 4 : neuron 4
    q_val_n5 = np.zeros((1,3)) #agent 5 : neuron 5
        
    start_scope()
    tau = 1*ms
    eqs = '''
    dv/dt = (1-v)/tau : 1
    '''

    #creating environment

    indices1 = array([0,1,2])
    times = array([0,0,0])*ms
    #input layer
    Gi = SpikeGeneratorGroup(3, indices1, times)
    #first layer
    G1 = NeuronGroup(2, eqs, threshold='v>1', reset='v = 0',method = 'exact')
    S1 = Synapses(Gi, G1, 'w : 1',on_pre='v += w',multisynaptic_index='k')
    S1.connect(i=[0,0,1,1,2,2],j=[0,1,0,1,0,1],n=6)
    S1.w['k<6'] = 0.1
    #S1.connect()

    #second layer
    G2 = NeuronGroup(1, eqs, threshold='v>1', reset='v = 0',method = 'exact')
    S2 = Synapses(G1,G2, 'w : 1',on_pre ='v +=w',multisynaptic_index='k')
    S2.connect(i=[0,1],j=[0,0],n=2)
    S2.w['k<2'] =  0.1
    #S2.connect()

    M = SpikeMonitor(Gi)
    M1 = SpikeMonitor(G1)
    M2 = SpikeMonitor(G2,variables=['t'])

    statemon1 = StateMonitor(G1, 'v', record=True)
    statemon2 = StateMonitor(G2, 'v', record=True)
    Sm = StateMonitor(S1,'w',record=[0,1])
    Sm2 = StateMonitor(S2,'w',record=[0,1])
    
    W_n1.append(weight)
    W_n2.append(weight)
    W_n3.append(weight)
    W_n4.append(weight)
    W_n5.append(weight)
    store()
    #continue taking actions
    while size(M2.i) != arr[1,3] :
        restore()
        action_index1 = get_next_action(q_val_n1,epsilon)
        action_index2 = get_next_action(q_val_n2,epsilon)
        action_index3 = get_next_action(q_val_n3,epsilon)
        action_index4 = get_next_action(q_val_n4,epsilon)
        action_index5 = get_next_action(q_val_n5,epsilon)
        #increase weight
        weight1 = W_n1[-1]
        weight2 = W_n2[-1]
        weight3 = W_n3[-1]
        weight4 = W_n4[-1]
        weight5 = W_n5[-1]
        if action_index1 == 0:
            #S1.connect(i=0,j=[0,1],n=2)
            S1.w[0] = 'weight1+W'
            S1.w[1] = 'weight1+W'
            W_n1.append(S1.w[0])
            
        if action_index2 == 0:
            #S1.connect(i=1,j=[0,1],n=2)
            S1.w[2] = 'weight2+W'
            S1.w[3] = 'weight2+W'
            W_n2.append(S1.w[2])
            
        if action_index3 == 0:
           # S1.connect(i=2,j=[0,1],n=2)
            S1.w[4] = 'weight3+W'
            S1.w[5] = 'weight3+W'
            W_n3.append(S1.w[4])
            
        if action_index4 == 0:
            #S2.connect(i=[0,1],j=[0,0],n=2)
            S2.w[0] = 'weight4+W'
            #S2.w['1'] = 'weight5'
            W_n4.append(S2.w[0])
            #W_n5.append(S2.w['1'])
            
        if action_index5 == 0:
            #S2.connect(i=[0,1],j=[0,0],n=2)
            S2.w[1] = 'weight5+W'
            W_n5.append(S2.w[1])
        
        #fixed weight 
        if action_index1 == 1:
            #S1.connect(i=0,j=[0,1],n=2)
            S1.w[0] = 'weight1'
            S1.w[1] = 'weight1'
            W_n1.append(S1.w[0])
        
        if action_index2 == 1:
            #S1.connect(i=1,j=[0,1],n=2)
            S1.w[2] = 'weight2'
            S1.w[3] = 'weight2'
            W_n2.append(S1.w[2])
            
        if action_index3 == 1:
            #S1.connect(i=2,j=[0,1],n=2)
            S1.w[4] = 'weight3'
            S1.w[5] = 'weight3'
            W_n3.append(S1.w[4])
        
        if action_index4 == 1 :
            #S2.connect(i=[0,1],j=[0,0],n=2)
            S2.w[0] = 'weight4'
            #S2.w['1'] = 'weight5'
            W_n4.append(S2.w[0])
            #W_n5.append(S2.w['1'])
        
        if action_index5 == 1:
         #   S2.connect(i=1,j=0,n=1)
            S2.w[1] = 'weight5'
            W_n5.append(S2.w[1])
            
        #decrease weight
        if action_index1 == 2:
            #S1.connect(i=0,j=[0,1],n=2)
            S1.w[0] = 'weight1-W'
            S1.w[1] = 'weight1-W'
            W_n1.append(S1.w[0])
            
        if action_index2 == 2:
            #S1.connect(i=1,j=[0,1],n=2)
            S1.w[2] = 'weight2-W'
            S1.w[3] = 'weight2-W'
            W_n2.append(S1.w[2])
            
        if action_index3 == 2:
            #S1.connect(i=2,j=[0,1])
            S1.w[4] = 'weight3-W'
            S1.w[5] = 'weight3-W'
            W_n3.append(S1.w[4])
            
        if action_index4 == 2:
            #S2.connect(i=[0,0],j=[0,0],n=2)
            S2.w[0] = 'weight4-W'
            W_n4.append(S2.w[0])
            
        if action_index5 == 2:
            #S2.connect(i=1,j=0,n=1)
            S2.w[1] = 'weight5-W'
            W_n5.append(S2.w[1])            
        run(1*ms)
        #perform the chosen action, and transition to the next state 
        real_output = size(M2.i)
        #receive the reward for moving to the new state, and calculate the temporal difference
        reward = get_reward(1,real_output)
        if S1.w[0] <0 or S1.w[1]<0 or S1.w[2]<0 or S1.w[3]<0 or S1.w[4] <0 or S1.w[5]<0 or S2.w[0]<0 or S2.w[1]<0 :
            reward = -100
        q_val_n1[0,action_index1] = q_val_n1[0,action_index1] + alpha*(reward+gamma*(max(q_val_n1[0]))-q_val_n1[0,action_index1])
        q_val_n2[0,action_index2] = q_val_n2[0,action_index2] + alpha*(reward+gamma*(max(q_val_n2[0]))-q_val_n2[0,action_index2])
        q_val_n3[0,action_index3] = q_val_n3[0,action_index3] + alpha*(reward+gamma*(max(q_val_n3[0]))-q_val_n3[0,action_index3])
        q_val_n4[0,action_index4] = q_val_n4[0,action_index4] + alpha*(reward+gamma*(max(q_val_n4[0]))-q_val_n4[0,action_index4])
        q_val_n5[0,action_index5] = q_val_n5[0,action_index5] + alpha*(reward+gamma*(max(q_val_n5[0]))-q_val_n5[0,action_index5])
        print('neuron1','q_table =',q_val_n1,'action_index=',action_index1,'reward =',reward,'real_output=',size(M2.i),S1.w[0])
        print('neuron2','q_table =',q_val_n2,'action_index=',action_index2,'reward =',reward,'real_output=',size(M2.i))
        print('neuron3','q_table =',q_val_n3,'action_index=',action_index3,'reward =',reward,'real_output=',size(M2.i))
        print('neuron4','q_table =',q_val_n4,'action_index=',action_index4,'reward =',reward,'real_output=',size(M2.i),S2.w[0])
        print('neuron5','q_table =',q_val_n5,'action_index=',action_index5,'reward =',reward,'real_output=',size(M2.i),S2.w[1])

    


neuron1 q_table = [[-0.9  0.   0. ]] action_index= 0 reward = -1 real_output= 0 0.1
neuron2 q_table = [[-0.9  0.   0. ]] action_index= 0 reward = -1 real_output= 0
neuron3 q_table = [[-0.9  0.   0. ]] action_index= 0 reward = -1 real_output= 0
neuron4 q_table = [[-0.9  0.   0. ]] action_index= 0 reward = -1 real_output= 0 0.1
neuron5 q_table = [[-0.9  0.   0. ]] action_index= 0 reward = -1 real_output= 0 0.1
neuron1 q_table = [[-0.9 -0.9  0. ]] action_index= 1 reward = -1 real_output= 0 0.1
neuron2 q_table = [[-0.9 -0.9  0. ]] action_index= 1 reward = -1 real_output= 0
neuron3 q_table = [[-0.9 -0.9  0. ]] action_index= 1 reward = -1 real_output= 0
neuron4 q_table = [[-0.9 -0.9  0. ]] action_index= 1 reward = -1 real_output= 0 0.1
neuron5 q_table = [[-0.9 -0.9  0. ]] action_index= 1 reward = -1 real_output= 0 0.1
neuron1 q_table = [[-0.9 -0.9 -0.9]] action_index= 2 reward = -1 real_output= 0 0.0
neuron2 q_table = [[-0.9 -0.9 -0.9]] action_index= 2 reward = -1 real_output= 0
neuron3 q_ta

neuron1 q_table = [[-277.77712029 -247.23675275 -247.17503529]] action_index= 0 reward = -100 real_output= 0 0.2
neuron2 q_table = [[-277.77712029 -248.28601748 -247.72438855]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-247.78610601 -247.72438855 -228.33137713]] action_index= 2 reward = -100 real_output= 0
neuron4 q_table = [[-277.77712029 -248.28601748 -247.17503529]] action_index= 0 reward = -100 real_output= 0 0.2
neuron5 q_table = [[-279.37138006 -246.57813253 -225.15039   ]] action_index= 0 reward = -100 real_output= 0 0.4
neuron1 q_table = [[-277.77712029 -247.23675275 -314.92928211]] action_index= 2 reward = -100 real_output= 0 0.1
neuron2 q_table = [[-277.77712029 -248.28601748 -315.42919358]] action_index= 2 reward = -100 real_output= 0
neuron3 q_table = [[-247.78610601 -247.72438855 -297.78155318]] action_index= 2 reward = -100 real_output= 0
neuron4 q_table = [[-277.77712029 -248.28601748 -314.92928211]] action_index= 2 reward = -100 real_output= 0 0.1

neuron1 q_table = [[-549.30970713 -530.25153607 -530.21302239]] action_index= 0 reward = -100 real_output= 0 0.2
neuron2 q_table = [[-549.30970713 -530.90631099 -530.55583648]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-530.59435016 -489.26700248 -518.45397396]] action_index= 0 reward = -100 real_output= 0
neuron4 q_table = [[-549.30970713 -530.90631099 -530.21302239]] action_index= 0 reward = -100 real_output= 0 0.2
neuron5 q_table = [[-550.30457648 -529.84053588 -516.46893574]] action_index= 0 reward = -100 real_output= 0 0.4
neuron1 q_table = [[-549.30970713 -530.25153607 -572.49385037]] action_index= 2 reward = -100 real_output= 0 0.1
neuron2 q_table = [[-549.30970713 -530.90631099 -572.8058112 ]] action_index= 2 reward = -100 real_output= 0
neuron3 q_table = [[-530.59435016 -535.23297225 -518.45397396]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-549.30970713 -530.90631099 -572.49385037]] action_index= 2 reward = -100 real_output= 0 0.1

neuron1 q_table = [[-718.75476977 -706.8618584  -706.83782462]] action_index= 0 reward = -100 real_output= 0 0.2
neuron2 q_table = [[-718.75476977 -692.73393947 -707.05175163]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-678.10525869 -700.65041956 -699.49980041]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-718.75476977 -692.69162166 -706.83782462]] action_index= 0 reward = -100 real_output= 0 0.2
neuron5 q_table = [[-691.6215387  -720.87398445 -698.26107275]] action_index= 1 reward = -100 real_output= 0 0.30000000000000004
neuron1 q_table = [[-734.41411492 -706.8618584  -706.83782462]] action_index= 0 reward = -100 real_output= 0 0.30000000000000004
neuron2 q_table = [[-718.75476977 -720.38788492 -707.05175163]] action_index= 1 reward = -100 real_output= 0
neuron3 q_table = [[-707.07578541 -700.65041956 -699.49980041]] action_index= 0 reward = -100 real_output= 0
neuron4 q_table = [[-718.75476977 -720.34937571 -706.83782462]] action_index= 1 

neuron1 q_table = [[-817.08059699 -817.07237668 -817.05737884]] action_index= 1 reward = -100 real_output= 0 0.20000000000000004
neuron2 q_table = [[-818.28383873 -808.25610113 -799.11085293]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-802.41989363 -794.72114196 -800.59446395]] action_index= 2 reward = -100 real_output= 0
neuron4 q_table = [[-824.49393568 -808.22969346 -817.05737884]] action_index= 0 reward = -100 real_output= 0 0.2
neuron5 q_table = [[-824.88135384 -808.5894437  -811.70520997]] action_index= 0 reward = -100 real_output= 0 0.4
neuron1 q_table = [[-817.08059699 -817.07237668 -833.52221474]] action_index= 2 reward = -100 real_output= 0 0.10000000000000003
neuron2 q_table = [[-818.28383873 -808.25610113 -817.19087617]] action_index= 2 reward = -100 real_output= 0
neuron3 q_table = [[-802.41989363 -813.19623918 -800.59446395]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-824.49393568 -825.48902104 -817.05737884]] action_index= 1 

neuron1 q_table = [[-885.85241256 -885.84728282 -885.83792368]] action_index= 0 reward = -100 real_output= 0 0.20000000000000004
neuron2 q_table = [[-875.38821328 -880.34564348 -885.92123031]] action_index= 2 reward = -100 real_output= 0
neuron3 q_table = [[-876.70366239 -871.89939387 -875.56453559]] action_index= 0 reward = -100 real_output= 0
neuron4 q_table = [[-890.4785742  -879.72177159 -880.32712977]] action_index= 0 reward = -100 real_output= 0 0.2
neuron5 q_table = [[-879.91245669 -881.92518142 -880.76732351]] action_index= 2 reward = -100 real_output= 0 0.30000000000000004
neuron1 q_table = [[-885.85241256 -885.84728282 -896.11251055]] action_index= 2 reward = -100 real_output= 0 0.10000000000000003
neuron2 q_table = [[-886.60327408 -880.34564348 -885.92123031]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-876.70366239 -883.42844842 -875.56453559]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-890.4785742  -890.54681214 -880.32712977]] 

neuron1 q_table = [[-921.72333643 -928.76503503 -928.75919463]] action_index= 1 reward = -100 real_output= 0 0.10000000000000003
neuron2 q_table = [[-918.41862858 -919.30144661 -914.03354745]] action_index= 1 reward = -100 real_output= 0
neuron3 q_table = [[-915.44958457 -922.88753774 -914.66842881]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-918.69935932 -918.70450154 -925.32028208]] action_index= 2 reward = -100 real_output= 0 0.10000000000000003
neuron5 q_table = [[-925.06151275 -919.03023921 -925.59497712]] action_index= 2 reward = -100 real_output= 0 0.30000000000000004
neuron1 q_table = [[-928.76823615 -928.76503503 -928.75919463]] action_index= 0 reward = -100 real_output= 0 0.20000000000000004
neuron2 q_table = [[-918.41862858 -919.30144661 -921.77052818]] action_index= 2 reward = -100 real_output= 0
neuron3 q_table = [[-915.44958457 -922.88753774 -922.34827022]] action_index= 2 reward = -100 real_output= 0
neuron4 q_table = [[-926.01641698 -918.70450154 

neuron1 q_table = [[-951.15284572 -951.15065055 -951.20565119]] action_index= 2 reward = -100 real_output= 0 0.10000000000000003
neuron2 q_table = [[-949.09060179 -944.12213297 -946.35417021]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-947.23782289 -943.0428251  -942.53945054]] action_index= 0 reward = -100 real_output= 0
neuron4 q_table = [[-949.2657868  -949.26899571 -944.7470509 ]] action_index= 1 reward = -100 real_output= 0 0.20000000000000004
neuron5 q_table = [[-953.23597505 -949.47226649 -948.97678458]] action_index= 0 reward = -100 real_output= 0 0.4
neuron1 q_table = [[-951.15284572 -955.547092   -951.20565119]] action_index= 1 reward = -100 real_output= 0 0.10000000000000003
neuron2 q_table = [[-949.09060179 -949.151141   -946.35417021]] action_index= 1 reward = -100 real_output= 0
neuron3 q_table = [[-947.23782289 -943.0428251  -947.71089999]] action_index= 2 reward = -100 real_output= 0
neuron4 q_table = [[-949.2657868  -949.26899571 -949.71981632]] 

neuron1 q_table = [[-969.51645222 -969.51643568 -966.53929433]] action_index= 1 reward = -100 real_output= 0 0.20000000000000004
neuron2 q_table = [[-965.08890003 -965.09346209 -966.52327776]] action_index= 2 reward = -100 real_output= 0
neuron3 q_table = [[-961.2577624  -964.45689197 -964.14277006]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-968.34022011 -965.2092749  -965.52038365]] action_index= 0 reward = -100 real_output= 0 0.20000000000000004
neuron5 q_table = [[-967.93158813 -968.46907007 -968.15987343]] action_index= 1 reward = -100 real_output= 0 0.30000000000000004
neuron1 q_table = [[-969.51645222 -969.84847198 -966.53929433]] action_index= 1 reward = -100 real_output= 0 0.20000000000000004
neuron2 q_table = [[-968.23089903 -965.09346209 -966.52327776]] action_index= 0 reward = -100 real_output= 0
neuron3 q_table = [[-961.2577624  -965.06447674 -964.14277006]] action_index= 1 reward = -100 real_output= 0
neuron4 q_table = [[-968.34022011 -968.34044016 

In [4]:
#izhikevich neuron model
a = 0.02/ms; b = 0.2/ms
eqs = '''dvm/dt = (0.04/ms/mV)*vm**2+(5/ms)*vm+140*mV/ms-w + I : volt
         dw/dt = a*(b*vm-w) : volt/second
         I : volt/second'''

In [2]:
import numpy as np

In [3]:
np.zeros((2,2))

array([[0., 0.],
       [0., 0.]])